In [5]:
import os
import codecs
print(os.getcwd())
data_directory = os.path.join('/Users/jmandivara/', 'Downloads',
                              'yelp_dataset_challenge_academic_dataset')

businesses_filepath = os.path.join(data_directory,
                                   'yelp_academic_dataset_business.json')

with codecs.open(businesses_filepath, encoding='utf_8') as f:
    first_business_record = f.readline() 

print(first_business_record)

/Users/jmandivara/Documents/TensorFlow/NLP
{"business_id":"0DI8Dt2PJp07XkVvIElIcQ","name":"Innovative Vapors","neighborhood":"","address":"227 E Baseline Rd, Ste J2","city":"Tempe","state":"AZ","postal_code":"85283","latitude":33.3782141,"longitude":-111.936102,"stars":4.5,"review_count":17,"is_open":0,"attributes":["BikeParking: True","BusinessAcceptsBitcoin: False","BusinessAcceptsCreditCards: True","BusinessParking: {'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}","DogsAllowed: False","RestaurantsPriceRange2: 2","WheelchairAccessible: True"],"categories":["Tobacco Shops","Nightlife","Vape Shops","Shopping"],"hours":["Monday 11:0-21:0","Tuesday 11:0-21:0","Wednesday 11:0-21:0","Thursday 11:0-21:0","Friday 11:0-22:0","Saturday 10:0-22:0","Sunday 11:0-18:0"],"type":"business"}



In [ ]:
#business_id and categories are ones to concentrate

In [6]:
review_json_filepath = os.path.join(data_directory,
                                    'yelp_academic_dataset_review.json')

with codecs.open(review_json_filepath, encoding='utf_8') as f:
    first_review_record = f.readline()
    
print(first_review_record)

{"review_id":"NxL8SIC5yqOdnlXCg18IBg","user_id":"KpkOkG6RIf4Ra25Lhhxf1A","business_id":"2aFiy99vNLklCx3T_tGS9A","stars":5,"date":"2011-10-10","text":"If you enjoy service by someone who is as competent as he is personable, I would recommend Corey Kaplan highly. The time he has spent here has been very productive and working with him educational and enjoyable. I hope not to need him again (though this is highly unlikely) but knowing he is there if I do is very nice. By the way, I'm not from El Centro, CA. but Scottsdale, AZ.","useful":0,"funny":0,"cool":0,"type":"review"}



In [70]:
import json

restaurant_ids = set()
i=0

# open the businesses file
with codecs.open(businesses_filepath, encoding='utf_8') as f:
    
    # iterate through each line (json record) in the file
    for business_json in f:
        
        # convert the json record to a Python dict
        business = json.loads(business_json)
        i=i+1;
        
        # if this business is not a restaurant, skip to the next one
        #print(business)
        #print(business[u'categories'])
        
        if business[u'categories']==None or u'Restaurants' not in business[u'categories']:
            continue
            
        # add the restaurant business id to our restaurant_ids set
        restaurant_ids.add(business[u'business_id'])

# turn restaurant_ids into a frozenset, as we don't need to change it anymore
restaurant_ids = frozenset(restaurant_ids)

# print the number of unique restaurant ids in the dataset
print('{:,}'.format(len(restaurant_ids)), u'restaurants in the dataset.')
print(i)

48,485 restaurants in the dataset.
144072


In [77]:
#intermediate_directory = os.path.join('..', '')

review_txt_filepath = os.path.join(os.getcwd(),
                                   'review_text_all.txt')

In [81]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
#review_count = 0
if 0 == 0:
    
    review_count = 0

    # create & open a new file in write mode
    with codecs.open(review_txt_filepath, 'w', encoding='utf_8') as review_txt_file:

        # open the existing review json file
        with codecs.open(review_json_filepath, encoding='utf_8') as review_json_file:

            # loop through all reviews in the existing file and convert to dict
            for review_json in review_json_file:
                review = json.loads(review_json)

                # if this review is not about a restaurant, skip to the next one
                if review[u'business_id'] not in restaurant_ids:
                    continue

                # write the restaurant review as a line in the new file
                # escape newline characters in the original review text
                review_txt_file.write(review[u'text'].replace('\n', '\\n') + '\n')
                review_count += 1

    print(u'''Text from {:,} restaurant reviews written to the new txt file.'''.format(review_count))
    
else:
    
    with codecs.open(review_txt_filepath, encoding='utf_8') as review_txt_file:
        for review_count, line in enumerate(review_txt_file):
            pass
        
    print(u'Text from {:,} restaurant reviews in the txt file.'.format(review_count + 1))

Text from 2,577,298 restaurant reviews written to the new txt file.
CPU times: user 2min 5s, sys: 7.56 s, total: 2min 12s
Wall time: 2min 17s


In [88]:
import spacy
import pandas as pd
import itertools as it

nlp = spacy.load('en')

In [89]:
with codecs.open(review_txt_filepath, encoding='utf_8') as f:
    sample_review = list(it.islice(f, 8, 9))[0]
    sample_review = sample_review.replace('\\n', '\n')
        
print(sample_review)

Was boozin and cruisin with my canine comrade. Went all the way out to Hudson via Brecksville/82. Looking for anything to Yelp. Nothing. Not even in the strip mall I used to hit in Hudson on my dry cleaning route back in the day. There was a deli, but it opened at fucking 11! Who even wants to go to a deli after 11? Weak. Breezed through Peninsula. Winking Lizard? Bah. Ended up backtracking through Brecksville and I remembered an old friend. London Pickle Works! Was it still there some four years after I stopped in while house shopping? Yup. Incidentally, investing in a condo down there probably would have made more sense than buying a house in Lakewood. Live and learn.

Cozied up to the bar and ordered a Killians, plus cheesesteak to go(for pooch). Like the Unicorn, food was amazingly done before I could chug my brew. I could get used to this kind of service. The bar was all old men. Matt, I don't know how you do it. They were watching Fox news for chrissakes!! Total was eight dollars

In [90]:
%%time
parsed_review = nlp(sample_review)
print (parsed_review)

Was boozin and cruisin with my canine comrade. Went all the way out to Hudson via Brecksville/82. Looking for anything to Yelp. Nothing. Not even in the strip mall I used to hit in Hudson on my dry cleaning route back in the day. There was a deli, but it opened at fucking 11! Who even wants to go to a deli after 11? Weak. Breezed through Peninsula. Winking Lizard? Bah. Ended up backtracking through Brecksville and I remembered an old friend. London Pickle Works! Was it still there some four years after I stopped in while house shopping? Yup. Incidentally, investing in a condo down there probably would have made more sense than buying a house in Lakewood. Live and learn.

Cozied up to the bar and ordered a Killians, plus cheesesteak to go(for pooch). Like the Unicorn, food was amazingly done before I could chug my brew. I could get used to this kind of service. The bar was all old men. Matt, I don't know how you do it. They were watching Fox news for chrissakes!! Total was eight dollars

In [91]:
for num, sentence in enumerate(parsed_review.sents):
    print ('Sentence {}:'.format(num + 1))
    print( sentence)
    print ('')

Sentence 1:
Was boozin and cruisin with my canine comrade.

Sentence 2:
Went all the way out to Hudson via Brecksville/82.

Sentence 3:
Looking for anything to Yelp.

Sentence 4:
Nothing.

Sentence 5:
Not even in the strip mall I used to hit in Hudson on my dry cleaning route back in the day.

Sentence 6:
There was a deli, but it opened at fucking 11!

Sentence 7:
Who even wants to go to a deli after 11?

Sentence 8:
Weak

Sentence 9:
. Breezed through Peninsula.

Sentence 10:
Winking Lizard?

Sentence 11:
Bah.

Sentence 12:
Ended up backtracking through Brecksville and I remembered an old friend.

Sentence 13:
London Pickle Works!

Sentence 14:
Was it still there some four years after I stopped in while house shopping?

Sentence 15:
Yup.

Sentence 16:
Incidentally, investing in a condo down there probably would have made more sense than buying a house in Lakewood.

Sentence 17:
Live and learn.



Sentence 18:
Cozied up to the bar and ordered a Killians, plus cheesesteak to go(for pooc

In [92]:
for num, entity in enumerate(parsed_review.ents):
    print('Entity {}:'.format(num + 1), entity, '-', entity.label_)
    print('')

Entity 1: Hudson - PERSON

Entity 2: Yelp - PERSON

Entity 3: Hudson - PERSON

Entity 4: the day - DATE

Entity 5: 11 - DATE

Entity 6: 11 - CARDINAL

Entity 7: Peninsula - PERSON

Entity 8: Brecksville - GPE

Entity 9: London Pickle Works - ORG

Entity 10: some four years - DATE

Entity 11: Lakewood - GPE

Entity 12: Live - ORG

Entity 13: Killians - NORP

Entity 14: Unicorn - GPE

Entity 15: Matt - PERSON

Entity 16: Fox - ORG

Entity 17: three - CARDINAL

Entity 18: Steak'um. - PERCENT

Entity 19: one - CARDINAL

Entity 20: one - CARDINAL

Entity 21: NES - ORG

Entity 22: Fries - PERSON

Entity 23: eight dollars - MONEY

Entity 24: four more years - DATE



In [96]:
token_text = [token.orth_ for token in parsed_review]
token_pos = [token.pos_ for token in parsed_review]

pd.DataFrame(list(zip(token_text, token_pos)),columns=['token_text', 'part_of_speech'])


,token_text,part_of_speech
0,Was,VERB
1,boozin,NOUN
2,and,CCONJ
3,cruisin,NOUN
4,with,ADP
5,my,ADJ
6,canine,ADJ
7,comrade,NOUN
8,.,PUNCT
9,Went,VERB


In [97]:
token_lemma = [token.lemma_ for token in parsed_review]
token_shape = [token.shape_ for token in parsed_review]

pd.DataFrame(list(zip(token_text, token_lemma, token_shape)),columns=['token_text', 'token_lemma', 'token_shape'])

,token_text,token_lemma,token_shape
0,Was,be,Xxx
1,boozin,boozin,xxxx
2,and,and,xxx
3,cruisin,cruisin,xxxx
4,with,with,xxxx
5,my,-PRON-,xx
6,canine,canine,xxxx
7,comrade,comrade,xxxx
8,.,.,.
9,Went,go,Xxxx


In [98]:
token_entity_type = [token.ent_type_ for token in parsed_review]
token_entity_iob = [token.ent_iob_ for token in parsed_review]

pd.DataFrame(list(zip(token_text, token_entity_type, token_entity_iob)),columns=['token_text', 'entity_type', 'inside_outside_begin'])

,token_text,entity_type,inside_outside_begin
0,Was,,O
1,boozin,,O
2,and,,O
3,cruisin,,O
4,with,,O
5,my,,O
6,canine,,O
7,comrade,,O
8,.,,O
9,Went,,O


In [99]:
token_attributes = [(token.orth_,
                     token.prob,
                     token.is_stop,
                     token.is_punct,
                     token.is_space,
                     token.like_num,
                     token.is_oov)
                    for token in parsed_review]

df = pd.DataFrame(token_attributes,
                  columns=['text',
                           'log_probability',
                           'stop?',
                           'punctuation?',
                           'whitespace?',
                           'number?',
                           'out of vocab.?'])

df.loc[:, 'stop?':'out of vocab.?'] = (df.loc[:, 'stop?':'out of vocab.?']
                                       .applymap(lambda x: u'Yes' if x else u''))
                                               
df

,text,log_probability,stop?,punctuation?,whitespace?,number?,out of vocab.?
0,Was,-9.937194,Yes,,,,
1,boozin,-19.579313,,,,,Yes
2,and,-4.195279,Yes,,,,
3,cruisin,-19.579313,,,,,Yes
4,with,-5.363765,Yes,,,,
5,my,-5.918125,Yes,,,,
6,canine,-19.579313,,,,,
7,comrade,-19.579313,,,,,
8,.,-3.072948,,Yes,,,
9,Went,-19.579313,,,,,


In [100]:
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence

In [101]:
def punct_space(token):
    """
    helper function to eliminate tokens
    that are pure punctuation or whitespace
    """
    
    return token.is_punct or token.is_space

def line_review(filename):
    """
    generator function to read in reviews from the file
    and un-escape the original line breaks in the text
    """
    
    with codecs.open(filename, encoding='utf_8') as f:
        for review in f:
            yield review.replace('\\n', '\n')
            
def lemmatized_sentence_corpus(filename):
    """
    generator function to use spaCy to parse reviews,
    lemmatize the text, and yield sentences
    """
    
    for parsed_review in nlp.pipe(line_review(filename),
                                  batch_size=10000, n_threads=4):
        
        for sent in parsed_review.sents:
            yield u' '.join([token.lemma_ for token in sent
                             if not punct_space(token)])

In [102]:
unigram_sentences_filepath = os.path.join(os.getcwd(),
                                          'unigram_sentences_all.txt')

In [ ]:
i=0
with codecs.open(unigram_sentences_filepath, 'w', encoding='utf_8') as f:
    for sentence in lemmatized_sentence_corpus(review_txt_filepath):
        f.write(sentence + '\n')
        i=i+1
        if(i%100==0):
            print(i)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

167900
168000
168100
168200
168300
168400
168500
168600
168700
168800
168900
169000
169100
169200
169300
169400
169500
169600
169700
169800
169900
170000
170100
170200
170300
170400
170500
170600
170700
170800
170900
171000
171100
171200
171300
171400
171500
171600
171700
171800
171900
172000
172100
172200
172300
172400
172500
172600
172700
172800
172900
173000
173100
173200
173300
173400
173500
173600
173700
173800
173900
174000
174100
174200
174300
174400
174500
174600
174700
174800
174900
175000
175100
175200
175300
175400
175500
175600
175700
175800
175900
176000
176100
176200
176300
176400
176500
176600
176700
176800
176900
177000
177100
177200
177300
177400
177500
177600
177700
177800
177900
178000
178100
178200
178300
178400
178500
178600
178700
178800
178900
179000
179100
179200
179300
179400
179500
179600
179700
179800
179900
180000
180100
180200
180300
180400
180500
180600
180700
180800
180900
181000
181100
181200
181300
181400
181500
181600
181700
181800
181900
182000
182100

333900
334000
334100
334200
334300
334400
334500
334600
334700
334800
334900
335000
335100
335200
335300
335400
335500
335600
335700
335800
335900
336000
336100
336200
336300
336400
336500
336600
336700
336800
336900
337000
337100
337200
337300
337400
337500
337600
337700
337800
337900
338000
338100
338200
338300
338400
338500
338600
338700
338800
338900
339000
339100
339200
339300
339400
339500
339600
339700
339800
339900
340000
340100
340200
340300
340400
340500
340600
340700
340800
340900
341000
341100
341200
341300
341400
341500
341600
341700
341800
341900
342000
342100
342200
342300
342400
342500
342600
342700
342800
342900
343000
343100
343200
343300
343400
343500
343600
343700
343800
343900
344000
344100
344200
344300
344400
344500
344600
344700
344800
344900
345000
345100
345200
345300
345400
345500
345600
345700
345800
345900
346000
346100
346200
346300
346400
346500
346600
346700
346800
346900
347000
347100
347200
347300
347400
347500
347600
347700
347800
347900
348000
348100

502800
502900
503000
503100
503200
503300
503400
503500
503600
503700
503800
503900
504000
504100
504200
504300
504400
504500
504600
504700
504800
504900
505000
505100
505200
505300
505400
505500
505600
505700
505800
505900
506000
506100
506200
506300
506400
506500
506600
506700
506800
506900
507000
507100
507200
507300
507400
507500
507600
507700
507800
507900
508000
508100
508200
508300
508400
508500
508600
508700
508800
508900
509000
509100
509200
509300
509400
509500
509600
509700
509800
509900
510000
510100
510200
510300
510400
510500
510600
510700
510800
510900
511000
511100
511200
511300
511400
511500
511600
511700
511800
511900
512000
512100
512200
512300
512400
512500
512600
512700
512800
512900
513000
513100
513200
513300
513400
513500
513600
513700
513800
513900
514000
514100
514200
514300
514400
514500
514600
514700
514800
514900
515000
515100
515200
515300
515400
515500
515600
515700
515800
515900
516000
516100
516200
516300
516400
516500
516600
516700
516800
516900
517000

623300
623400
623500
623600
623700
623800
623900
624000
624100
624200
624300
624400
624500
624600
624700
624800
624900
625000
625100
625200
625300
625400
625500
625600
625700
625800
625900
626000
626100
626200
626300
626400
626500
626600
626700
626800
626900
627000
627100
627200
627300
627400
627500
627600
627700
627800
627900
628000
628100
628200
628300
628400
628500
628600
628700
628800
628900
629000
629100
629200
629300
629400
629500
629600
629700
629800
629900
630000
630100
630200
630300
630400
630500
630600
630700
630800
630900
631000
631100
631200
631300
631400
631500
631600
631700
631800
631900
632000
632100
632200
632300
632400
632500
632600
632700
632800
632900
633000
633100
633200
633300
633400
633500
633600
633700
633800
633900
634000
634100
634200
634300
634400
634500
634600
634700
634800
634900
635000
635100
635200
635300
635400
635500
635600
635700
635800
635900
636000
636100
636200
636300
636400
636500
636600
636700
636800
636900
637000
637100
637200
637300
637400
637500

...

6106900
6107000
6107100
6107200
6107300
6107400
6107500
6107600
6107700
6107800
6107900
6108000
6108100
6108200
6108300
6108400
6108500
6108600
6108700
6108800
6108900
6109000
6109100
6109200
6109300
6109400
6109500
6109600
6109700
6109800
6109900
6110000
6110100
6110200
6110300
6110400
6110500
6110600
6110700
6110800
6110900
6111000
6111100
6111200
6111300
6111400
6111500
6111600
6111700
6111800
6111900
6112000
6112100
6112200
6112300
6112400
6112500
6112600
6112700
6112800
6112900
6113000
6113100
6113200
6113300
6113400
6113500
6113600
6113700
6113800
6113900
6114000
6114100
6114200
6114300
6114400
6114500
6114600
6114700
6114800
6114900
6115000
6115100
6115200
6115300
6115400
6115500
6115600
6115700
6115800
6115900
6116000
6116100
6116200
6116300
6116400
6116500
6116600
6116700
6116800
6116900
6117000
6117100
6117200
6117300
6117400
6117500
6117600
6117700
6117800
6117900
6118000
6118100
6118200
6118300
6118400
6118500
6118600
6118700
6118800
6118900
6119000
6119100
6119200
6119300


6225200
6225300
6225400
6225500
6225600
6225700
6225800
6225900
6226000
6226100
6226200
6226300
6226400
6226500
6226600
6226700
6226800
6226900
6227000
6227100
6227200
6227300
6227400
6227500
6227600
6227700
6227800
6227900
6228000
6228100
6228200
6228300
6228400
6228500
6228600
6228700
6228800
6228900
6229000
6229100
6229200
6229300
6229400
6229500
6229600
6229700
6229800
6229900
6230000
6230100
6230200
6230300
6230400
6230500
6230600
6230700
6230800
6230900
6231000
6231100
6231200
6231300
6231400
6231500
6231600
6231700
6231800
6231900
6232000
6232100
6232200
6232300
6232400
6232500
6232600
6232700
6232800
6232900
6233000
6233100
6233200
6233300
6233400
6233500
6233600
6233700
6233800
6233900
6234000
6234100
6234200
6234300
6234400
6234500
6234600
6234700
6234800
6234900
6235000
6235100
6235200
6235300
6235400
6235500
6235600
6235700
6235800
6235900
6236000
6236100
6236200
6236300
6236400
6236500
6236600
6236700
6236800
6236900
6237000
6237100
6237200
6237300
6237400
6237500
6237600


6341000
6341100
6341200
6341300
6341400
6341500
6341600
6341700
6341800
6341900
6342000
6342100
6342200
6342300
6342400
6342500
6342600
6342700
6342800
6342900
6343000
6343100
6343200
6343300
6343400
6343500
6343600
6343700
6343800
6343900
6344000
6344100
6344200
6344300
6344400
6344500
6344600
6344700
6344800
6344900
6345000
6345100
6345200
6345300
6345400
6345500
6345600
6345700
6345800
6345900
6346000
6346100
6346200
6346300
6346400
6346500
6346600
6346700
6346800
6346900
6347000
6347100
6347200
6347300
6347400
6347500
6347600
6347700
6347800
6347900
6348000
6348100
6348200
6348300
6348400
6348500
6348600
6348700
6348800
6348900
6349000
6349100
6349200
6349300
6349400
6349500
6349600
6349700
6349800
6349900
6350000
6350100
6350200
6350300
6350400
6350500
6350600
6350700
6350800
6350900
6351000
6351100
6351200
6351300
6351400
6351500
6351600
6351700
6351800
6351900
6352000
6352100
6352200
6352300
6352400
6352500
6352600
6352700
6352800
6352900
6353000
6353100
6353200
6353300
6353400


6455400
6455500
6455600
6455700
6455800
6455900
6456000
6456100
6456200
6456300
6456400
6456500
6456600
6456700
6456800
6456900
6457000
6457100
6457200
6457300
6457400
6457500
6457600
6457700
6457800
6457900
6458000
6458100
6458200
6458300
6458400
6458500
6458600
6458700
6458800
6458900
6459000
6459100
6459200
6459300
6459400
6459500
6459600
6459700
6459800
6459900
6460000
6460100
6460200
6460300
6460400
6460500
6460600
6460700
6460800
6460900
6461000
6461100
6461200
6461300
6461400
6461500
6461600
6461700
6461800
6461900
6462000
6462100
6462200
6462300
6462400
6462500
6462600
6462700
6462800
6462900
6463000
6463100
6463200
6463300
6463400
6463500
6463600
6463700
6463800
6463900
6464000
6464100
6464200
6464300
6464400
6464500
6464600
6464700
6464800
6464900
6465000
6465100
6465200
6465300
6465400
6465500
6465600
6465700
6465800
6465900
6466000
6466100
6466200
6466300
6466400
6466500
6466600
6466700
6466800
6466900
6467000
6467100
6467200
6467300
6467400
6467500
6467600
6467700
6467800


6596200
6596300
6596400
6596500
6596600
6596700
6596800
6596900
6597000
6597100
6597200
6597300
6597400
6597500
6597600
6597700
6597800
6597900
6598000
6598100
6598200
6598300
6598400
6598500
6598600
6598700
6598800
6598900
6599000
6599100
6599200
6599300
6599400
6599500
6599600
6599700
6599800
6599900
6600000
6600100
6600200
6600300
6600400
6600500
6600600
6600700
6600800
6600900
6601000
6601100
6601200
6601300
6601400
6601500
6601600
6601700
6601800
6601900
6602000
6602100
6602200
6602300
6602400
6602500
6602600
6602700
6602800
6602900
6603000
6603100
6603200
6603300
6603400
6603500
6603600
6603700
6603800
6603900
6604000
6604100
6604200
6604300
6604400
6604500
6604600
6604700
6604800
6604900
6605000
6605100
6605200
6605300
6605400
6605500
6605600
6605700
6605800
6605900
6606000
6606100
6606200
6606300
6606400
6606500
6606600
6606700
6606800
6606900
6607000
6607100
6607200
6607300
6607400
6607500
6607600
6607700
6607800
6607900
6608000
6608100
6608200
6608300
6608400
6608500
6608600
